In [ ]:
from result_utils import *
import pandas as pd 
import wandb
import matplotlib.pyplot as plt
import os
import yaml
import zipfile
import json

In [ ]:
entity = "praca-inzynierska"
project = "final-experiments" 


api = wandb.Api()
runs = api.runs(f"{entity}/{project}")

In [ ]:
sum_runtime = 0
for run in runs:
    runtime = json.loads(run.summary._json_dict).get("_runtime")
    if runtime is not None:
        sum_runtime += float(runtime)
print(f"Total runtime (s): {sum_runtime}")
print(f"Total runtime (h): {sum_runtime/3600}")

In [ ]:
def zip_folder(folder_path, zip_name):
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                full_path = os.path.join(root, file)
                arcname = os.path.relpath(full_path, folder_path)
                zipf.write(full_path, arcname)

In [ ]:
metrics_max_in_step = ["train/gpu_mem_peak_mb"]
metrics_ignore = ["train/gpu_mem_peak_mb","train/update_skipped", "train/is_update_step","train/accum_step",
                  "train/lr","train/loss","train/grad_norm"]

def save_run_history_wide_csv_and_max_txt(run, base_dir):
    run_dir_name = f"{run.name}_{run.id}"
    run_dir = os.path.join(base_dir, run_dir_name)
    os.makedirs(run_dir, exist_ok=True)

    history = run.history(samples=100000)
    metric_columns = [c for c in history.columns if not c.startswith("_") and c not in metrics_ignore]

    if metric_columns:
        wide_df = history[["_step", "_runtime"] + metric_columns].copy()

        wide_df = wide_df.dropna(subset=metric_columns, how="all")

        csv_path = os.path.join(run_dir, "history_all_metrics.csv.gz")
        wide_df.to_csv(csv_path, index=False, na_rep='', compression='gzip')
        print(f"Saved wide CSV (gzip) for run {run.name} ({run.id})")
    else:
        print(f"No metrics to save for run {run.name} ({run.id})")

    for m in metrics_max_in_step:
        txt_path = os.path.join(run_dir, f"gpu_mem_and_runtime.txt")
        if m in history.columns:
            max_val = history[m].max(skipna=True)
        else:
            max_val = "N/A"

        try:
            runtime = json.loads(run.summary._json_dict).get("_runtime")
            if isinstance(runtime, str):
                runtime = float(runtime)
        except Exception:
            runtime = "N/A"

        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(f"peak_gpu_mem_mb: {max_val}\n")
            f.write(f"runtime_s: {runtime}\n")

    try:
        config_file = run.file("config.yaml")
        config_file.download(root=run_dir, replace=True)
    except Exception:
        print(f"config.yaml not found for run {run.name} ({run.id})")


In [ ]:
for run in runs:
    save_run_history_wide_csv_and_max_txt(run, './experiments')

In [ ]:
zip_folder('./experiments', 'experiments.zip')

In [ ]:
file_path = "./experiments/E1_pretraining_wikipedia_bertsmall_mha_hd228t3k/history_all_metrics.csv.gz"

df = pd.read_csv(file_path)

# df['test/accuracy'].loc[~df['test/accuracy'].isna()] 

In [ ]:
df.head()